# Prompt Search Testing

Idea: You start with a text (ex a page from a book), prompt(s), and hoped-for output(s).
From the prompt and long text, generate multiple outputs, and order them by semantic similarity with the 'correct' outputs. Then, modify the prompt in various ways and repeat.
Ways to modify the prompt:
	- Generate synonyms of the summary prompt and try the same thing. In other words, reformulate the prompt.
	- Generate infills for various [text, output] pairs to find prompts that are likely to generate the output.
	- Remove various words in the prmpt and infill multiple times.

Basically, build a general algorithm that takes prompt(s) for a task, possibly input(s), and wanted outputs, and generate better prompts for the task.

Function takes in the tuple ([prompt_11, prompt_12, ...], input_text_1, [desired_output_11, desired_output_12,...]), and outputs a list of prompts.

Tasks this might be useful for: 
	- Question answering (accuracy)
	- Summarization
	- Info extraction
	- Translation

In [ ]:
# result = openai.Embedding.create(model=EMBEDDING_MODEL, input=text)
# answer = openai.Completion.create(prompt=prompt, **COMPLETIONS_API_PARAMS)["choices"][0]["text"].strip(" \n")

## Imports

In [2]:
from typing import List, Dict, Tuple, Optional, Union, Any
import openai
import tokenize

import config

In [3]:
openai.api_key = config.OPENAI_API_KEY

## Helpers

In [4]:
def reformulate(prompt: str):
    raise NotImplementedError

def infill_prompt(text: str, outputs: List[str]):
    raise NotImplementedError

def modify_prompt(prompt: str):
    raise NotImplementedError

## Prompt Searcher

In [5]:
"""
Idea: You start with a text (ex a page from a book), prompt(s), and hoped-for output(s).
From the prompt and long text, generate multiple outputs, and order them by semantic similarity with the 'correct' outputs. Then, modify the prompt in various ways and repeat.
Ways to modify the prompt:
	- Generate synonyms of the summary prompt and try the same thing. In other words, reformulate the prompt.
	- Generate infills for various [text, output] pairs to find prompts that are likely to generate the output.
	- Modify the prompt in various ways (change single words for other synonymous words, etc).

Basically, build a general algorithm that takes prompt(s) for a task, possibly input(s), and wanted outputs, and generate better prompts for the task.

Function takes in the tuple ([prompt_11, prompt_12, ...], input_text_1, [desired_output_11, desired_output_12,...]), and outputs a list of prompts.

Tasks this might be useful for: 
	- Question answering (accuracy)
	- Summarization
	- Info extraction
	- Translation
 """

class PromptSearcher:
    def __init__(
            self,
            output_model: str = "code-davinci-002",
            reformulation_model: str = "gpt-3.5-turbo",
            embedding_model: str = "text-embedding-ada-002",
            endpoint: str = "https://api.openai.com/v1/completions",
            apikey: str = config.apikey
    ):
        self.output_model = output_model
        self.reformulation_model = reformulation_model
        self.embedding_model = embedding_model
        self.endpoint = endpoint
        self.apikey = apikey
    
    def reformulate(
        self,
        prompt: str,
        num_outputs: int = 5
    ) -> List[str]:
        """
        Reformulate the prompt using the reformulation model.

        Args:
            prompt (str): The prompt to reformulate.
            num_outputs (int, optional): The number of reformulated outputs. Defaults to 5.

        Returns:
            List[str]: The reformulated prompts.
        """
        raise NotImplementedError        
        
        # # Build the reformulation prompt
        # reformulation_prompt = f"Reformulate the following text. Try to modify it considerably while keeping all the same information.\n\nText: ```{prompt}```\n\nReformulated text:"
        
        # # Generate the reformulated prompts
        # reformulated_prompts = openai.Completion.create(
        #     prompt=reformulation_prompt,
        #     model=self.reformulation_model,
        #     max_tokens=100,
        #     temperature=0.5,
        #     top_p=1,
        #     frequency_penalty=0,
        #     presence_penalty=0,
        #     stop=["\n\n"]
        # )["choices"][:num_outputs]
        
        # # Return the reformulated prompts
        # return [choice["text"].strip(" \n") for choice in reformulated_prompts]
    
    def prompt_search(
        self,
        text: str,
        prompts: List[str],
        desired_outputs: List[str]
    ):
        raise NotImplementedError

AttributeError: module 'config' has no attribute 'apikey'

# This is code unrelated to the rest. Ignore it.

In [51]:
import requests
import json
import datetime
from typing import List, Tuple

In [177]:
def get_comments_times(market_link):
    # Returns a list of tuples of the form (comment, datetime)
    slug = market_link.split("/")[-1]
    API_get_slug = f"https://manifold.markets/api/v0/slug/{slug}"
    market_info = requests.get(API_get_slug)
    ID = market_info.json()['id']
    
    API_get_comments = f"https://manifold.markets/api/v0/comments"
    response = requests.get(API_get_comments, params={"contractId": ID})
    comments = response.json()
    comments_info: List[Tuple[str, str]] = []
    for comment in comments:
        # The problem: Comments are structured as a tree. We must find the text from each comment.
        comment_time = datetime.datetime.fromtimestamp(comment["createdTime"]/1000).strftime("%Y-%m-%d %H:%M:%S.%f")
        try:
            comment_info = comment["content"]["content"][0]["content"]
        except KeyError:
            try:
                comments_info.append((comment["text"], comment_time))
                continue
            except KeyError:
                if comment["content"]["content"][0]["type"] == "image":
                    comments_info.append(("[IMAGE]", comment_time))
                    continue
                elif comment["content"]["content"][0]["type"] == "iframe":
                    comments_info.append(("[IFRAME]", comment_time))
                    continue
                print(f'ERROR: {comment}')
                raise KeyError
            
        type_1 = comment_info[0]["type"]
        if type_1 == "text":
            comment_text = comment_info[0]["text"]
        elif type_1 == "mention":
            try:
                comment_text = comment_info[1]["text"]
            except IndexError:
                comment_text = "[MENTION]"
        comments_info.append((comment_text, comment_time))
    return comments_info

In [178]:
market_link = 'https://manifold.markets/NathanpmYoung/will-biden-be-the-2024-democratic-n'
comment_times = get_comments_times(market_link)

In [179]:
for comment in comment_times:
    print(comment)

(' See you at the annual wellness rave', '2023-03-05 22:55:45.206000')
(" That's totally fair, I was just wondering whether there was any sudden news I missed. May the odds be ever in your favor, and may Marianne Williamson become President!", '2023-03-05 03:53:31.037000')
(' Not interested enough in following prediction markets to make small, safe, long term bets, I would rather have a big bold bet that gets the lizard brain going+ ', '2023-03-05 03:50:05.777000')
(' What was the reason for that big NO bet?', '2023-03-05 02:42:09.878000')
(' yeah my dad got it this year and he was fine', '2023-03-03 23:56:49.961000')
(' I disagree. BCC is almost never lethal, and the lifetime risk of developing it among white people is 1 in 3. By some measures it is the most commonly diagnosed cancer in the US. This sort of thing is as common as dirt among elderly people like Mr. President.', '2023-03-03 21:14:00.732000')
(' I think this is enough to justify 1 or 2 pts lower, imo ', '2023-03-03 16:31:

In [96]:
def post_comment(market_link, comment, AUTHORIZATION_KEY):
    # Post a comment to a market
    # market_link: The link to the market
    # comment: The comment to post (markdown)
    # AUTHORIZATION_KEY: The authorization key for the Manifold API
    slug = market_link.split("/")[-1]
    API_get_slug = f"https://manifold.markets/api/v0/slug/{slug}"
    market_info = requests.get(API_get_slug)
    ID = market_info.json()['id']
    
    API_POST_URL = "https://manifold.markets/api/v0/comment"
    API_POST_HEADERS = {'Content-Type': 'application/json', 'Authorization': f'Key {AUTHORIZATION_KEY}'}
    API_POST_DATA = {"contractId": ID, "markdown": comment}
    
    response = requests.post(API_POST_URL, headers=API_POST_HEADERS, data=json.dumps(API_POST_DATA))
    return response


In [95]:
# market_link = "https://manifold.markets/firstuserhere/test-market-4-fe4ad98ccc1f"
# comment = "Yay it worked! (this is still a test) :)"
# AUTHORIZATION_KEY = "YOUR_AUTHORIZATION_KEY"

# post_comment(market_link, comment, AUTHORIZATION_KEY)

<Response [200]>

In [91]:
for comment_info in comment_times:
    print(comment_info)

(' but that can be faked so', '2023-03-08 23:26:43.168000')
(' the comment itself might include time', '2023-03-08 23:23:34.167000')
('Is there a way to know the precise second someone sent a comment on the API? Writing code to send a comment at the right second at close is easy, but how will you know if someone sent it a second earlier than close?', '2023-03-08 23:16:20.296000')
(' thats nerd shit that I have no idea about. 😁 ', '2023-03-08 21:46:57.554000')
('https://docs.manifold.markets/api#post-v0comment', '2023-03-08 21:42:33.547000')
(' wdym', '2023-03-08 21:41:31.676000')
("/ who's willing to schedule an API call for $1", '2023-03-08 21:35:35.960000')
(' and it aint me, thats for sure', '2023-03-08 20:35:35.175000')
('Seems like the real question is, "who is willing to stay up until 4am for 50 mana?"', '2023-03-08 20:35:16.543000')
("Note that you didn't ask about after, but I also won't do that. :)", '2023-03-08 20:31:40.668000')
(' Nope.', '2023-03-08 20:30:59.666000')
('Will